#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable


Распакуем наши данные из архива.

In [3]:
#!unzip ...

Загрузим данные из csv файла в датафрейм.

In [4]:
df = pd.read_csv('/Users/bonis/Downloads/input_data.csv', sep=';')
print(df.columns)

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id'],
      dtype='object')


Отобразим на карте координаты наших построек.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs): folium.Marker(location=[point[0], point[1]]).add_to(m)

print(df.columns)

display(m)

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id'],
      dtype='object')


/Users/bonis/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
# Расчёт дистанция до центров
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0088
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

MOSCOW = (55.755826, 37.617300)
SPB    = (59.934280, 30.335099)

# Добавляем новые признаки
df["dist_to_moscow_km"] = haversine_distance(df["geo_lat"], df["geo_lon"], MOSCOW[0], MOSCOW[1])
df["dist_to_spb_km"] = haversine_distance(df["geo_lat"], df["geo_lon"], SPB[0], SPB[1])

df["is_Moscow"] = df["dist_to_moscow_km"] < df["dist_to_spb_km"]
df["is_Saint_Peterburg"] = df["dist_to_spb_km"] < df["dist_to_moscow_km"]

df["in_20km_Moscow"] = df["dist_to_moscow_km"] <= 20
df["in_20km_SPB"] = df["dist_to_spb_km"] <= 20

print(df.columns)

display(df[["geo_lat","geo_lon","dist_to_moscow_km","dist_to_spb_km",
            "is_Moscow","is_Saint_Peterburg","in_20km_Moscow","in_20km_SPB"]].head())
print("Сводка по 20 км:", df[["in_20km_Moscow","in_20km_SPB"]].sum().to_dict())


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id', 'dist_to_moscow_km',
       'dist_to_spb_km', 'is_Moscow', 'is_Saint_Peterburg', 'in_20km_Moscow',
       'in_20km_SPB'],
      dtype='object')


,geo_lat,geo_lon,dist_to_moscow_km,dist_to_spb_km,is_Moscow,is_Saint_Peterburg,in_20km_Moscow,in_20km_SPB
0,56.780112,60.699355,1422.992237,1788.941034,True,False,False,False
1,44.608154,40.138381,1252.279418,1825.516829,True,False,False,False
2,55.540060,37.725112,24.927524,655.865220,True,False,False,False
3,44.608154,40.138381,1252.279418,1825.516829,True,False,False,False
4,44.738685,37.713668,1225.070771,1759.879265,True,False,False,False


Сводка по 20 км: {'in_20km_Moscow': 1027331, 'in_20km_SPB': 875966}


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [ ]:
# удаляем признаки, не несущие ценности для модели
to_drop = ["geo_lat","geo_lon","object_type","postal_code","street_id","id_region","house_id"]
df = df.drop(columns=[c for c in to_drop if c in df.columns])
df.head()

print(df.columns)


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'building_type', 'dist_to_moscow_km', 'dist_to_spb_km', 'is_Moscow',
       'is_Saint_Peterburg', 'in_20km_Moscow', 'in_20km_SPB'],
      dtype='object')


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

target = "price"

# определем категориальные и числовые
cat_cols = ["building_type", "is_Moscow", "is_Saint_Peterburg"]
print(df.columns)
num_cols = ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow_km', 'dist_to_spb_km', 'in_20km_Moscow', 'in_20km_SPB']

print("Категориальные:", cat_cols)
print("Числовые:", num_cols, f"всего {len(num_cols)}")

# onehot кодирование категор. признаков с добавлением колонок в df
if cat_cols:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_cat = pd.DataFrame(
        ohe.fit_transform(df[cat_cols]),
        columns=ohe.get_feature_names_out(cat_cols),
        index=df.index
    )
    df = pd.concat([df.drop(columns=cat_cols), X_cat], axis=1)

df.head()


Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'building_type', 'dist_to_moscow_km', 'dist_to_spb_km', 'is_Moscow',
       'is_Saint_Peterburg', 'in_20km_Moscow', 'in_20km_SPB'],
      dtype='object')
Категориальные: ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
Числовые: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow_km', 'dist_to_spb_km', 'in_20km_Moscow', 'in_20km_SPB'] всего 11


,date,price,level,levels,rooms,area,kitchen_area,dist_to_moscow_km,dist_to_spb_km,in_20km_Moscow,...,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Peterburg_False,is_Saint_Peterburg_True
0,2021-01-01,2451300,15,31,1,30.3,0.0,1422.992237,1788.941034,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1252.279418,1825.516829,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,24.927524,655.865220,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1252.279418,1825.516829,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,1225.070771,1759.879265,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Добавление двух признаков
if "date" in df.columns and np.issubdtype(df["date"].dtype, np.datetime64):
    dmin, dmax = df["date"].min(), df["date"].max()
    df["days_since_start"] = (df["date"] - dmin).dt.days
    df["days_to_end"]      = (dmax - df["date"]).dt.days
    df = df.drop(columns=["date"])

target = "price"
num_cols = [c for c in df.select_dtypes(include=["number"]).columns if c != target]

# Нормирование числовых признаков
scaler = StandardScaler()
scaled = scaler.fit_transform(df[num_cols].fillna(df[num_cols].median()))
df[[c + "_scaled" for c in num_cols]] = scaled

print("Скейлено числовых:", len(num_cols))
df[[*num_cols[:5], *(c+"_scaled" for c in num_cols[:5])]].head()


Скейлено числовых: 18


,level,levels,rooms,area,kitchen_area,level_scaled,levels_scaled,rooms_scaled,area_scaled,kitchen_area_scaled
0,15,31,1,30.3,0.0,1.622769,2.665027,-0.621470,-0.840577,0.082486
1,5,5,1,33.0,6.0,-0.270043,-0.936859,-0.621470,-0.741051,0.267565
2,4,13,3,85.0,12.0,-0.459324,0.171414,1.106234,1.175756,0.452644
3,3,5,3,82.0,9.0,-0.648605,-0.936859,1.106234,1.065171,0.360105
4,2,3,1,30.0,9.0,-0.837886,-1.213927,-0.621470,-0.851636,0.360105


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
import numpy as np

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean', weights='distance', p=2):
        self.k = int(n_neighbors)
        self.metric = metric
        self.weights = weights
        self.p = p
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        # Обучение для KNN
        # Приводим к float32
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float64).reshape(-1)
        assert X.shape[0] == y.shape[0]
        self.X_train, self.y_train = X, y
        return self

    def predict(self, X, block_size=1024):
        # Берём маленький блок теста (например, 512 строк), считаем расстояния до train, выдаём предсказания
        assert self.X_train is not None, "Сначала fit"
        A = np.asarray(X, dtype=np.float32)
        B = self.X_train
        ytr = self.y_train
        n_test, n_train = A.shape[0], B.shape[0]
        k = max(1, min(self.k, n_train)) # k не больше числа обучающих
        eps = 1e-12 # чтобы не делить на 0 при весах

        # Предрассчитывается ||B||^2 один раз, это ускоряет формулу расстояний
        B2 = np.sum(B * B, axis=1)[None, :]

        preds = np.empty(n_test, dtype=np.float32)
        for s in range(0, n_test, block_size):
            e = min(s + block_size, n_test) # [s:e) — границы текущего блока
            Ab = A[s:e] # берётся подматрицу теста
            A2 = np.sum(Ab * Ab, axis=1)[:, None] # ||A||^2 для каждой строки блока

            # D = sqrt( ||A||^2 + ||B||^2 - 2·A·B^T ) - матрица расстояний блока к train
            D = A2 + B2 - 2.0 * (Ab @ B.T)
            np.maximum(D, 0.0, out=D)
            D = np.sqrt(D, dtype=np.float32)

            # Берутся индексы k ближайших без полной сортировки
            kk = min(k, D.shape[1]-1) if D.shape[1] > 1 else 1
            idx = np.argpartition(D, kth=kk, axis=1)[:, :k]
            d_knn = np.take_along_axis(D, idx, axis=1)
            y_knn = ytr[idx]

            if self.weights == 'uniform':
                # Равные веса
                preds[s:e] = y_knn.mean(axis=1)
            elif self.weights == 'distance':
                # Веса 1/(d+eps): ближние соседи значат больше
                w = 1.0 / (d_knn + eps)
                # Если есть нулевые расстояния, то берётся точное значение
                zero = d_knn < eps
                pr = (w * y_knn).sum(axis=1) / w.sum(axis=1)
                if np.any(zero.any(axis=1)):
                    m = zero.any(axis=1)
                    pr[m] = (y_knn[m] * zero[m]).sum(axis=1) / zero[m].sum(axis=1)
                preds[s:e] = pr
            else:
                raise ValueError("weights must be 'uniform' или 'distance'")

        # Возвращается в float64, чтобы метрики из sklearn не ругались на типы
        return preds.astype(np.float64)

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.0,
                 max_iter=1000, momentum=0.9, batch_size=None, random_state=42):
        self.learning_rate = float(learning_rate) # скорость обучения
        self.optimization = optimization # какой метод оптимизации используем
        self.epsilon = float(epsilon) # маленькая константа, чтобы не делить на 0
        self.decay_rate = float(decay_rate) 
        self.max_iter = int(max_iter) # сколько проходов по данным делаем максимум
        self.momentum = float(momentum) # коэффициент инерции для Momentum
        self.batch_size = batch_size # размер мини-батча
        self.random_state = random_state # фиксируем генератор
        self.weights = None 
        self.bias = None
        self._v_w = None
        self._v_b = None
        self._G_w = None
        self._G_b = None

    # гарантируем, что X — матрица
    def _to_2d(self, X):
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        return X

    # Генератор индексов мини-батчей
    def _batches(self, n):
        if self.batch_size is None or self.batch_size >= n:
            yield np.arange(n)
        else:
            rng = np.random.default_rng(self.random_state)
            idx = np.arange(n)
            rng.shuffle(idx)
            for s in range(0, n, self.batch_size):
                yield idx[s:s+self.batch_size]

    # обучение
    def fit(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        n, d = X.shape
        # лениво инициализируем параметры
        if self.weights is None:
            self.weights = np.zeros(d, dtype=float)
        if self.bias is None:
            self.bias = 0.0
        # подготавливаем служебные переменные под выбранную оптимизацию
        if self.optimization == 'Momentum':
            self._v_w = np.zeros_like(self.weights)
            self._v_b = 0.0
        if self.optimization == 'AdaGrad':
            self._G_w = np.zeros_like(self.weights)
            self._G_b = 0.0

        prev_loss = np.inf
        lr0 = self.learning_rate

        # основной цикл обучения
        for t in range(self.max_iter):
            lr_t = lr0 / (1.0 + self.decay_rate * t)
            for idx in self._batches(n):
                Xb = X[idx]; yb = y[idx]; m = Xb.shape[0]
                y_pred = Xb @ self.weights + self.bias
                err = y_pred - yb
                grad_w = (2.0/m) * (Xb.T @ err)
                grad_b = (2.0/m) * err.sum()

                if self.optimization == 'SGD':
                    self.weights -= lr_t * grad_w
                    self.bias    -= lr_t * grad_b
                elif self.optimization == 'Momentum':
                    self._v_w = self.momentum * self._v_w + lr_t * grad_w
                    self._v_b = self.momentum * self._v_b + lr_t * grad_b
                    self.weights -= self._v_w
                    self.bias    -= self._v_b
                elif self.optimization == 'AdaGrad':
                    self._G_w += grad_w**2
                    self._G_b += grad_b**2
                    self.weights -= (lr_t / (np.sqrt(self._G_w) + self.epsilon)) * grad_w
                    self.bias    -= (lr_t / (np.sqrt(self._G_b) + self.epsilon)) * grad_b
                else:
                    raise ValueError("optimization must be 'SGD', 'Momentum', or 'AdaGrad'")

            loss = self.mse(X, y)
            if abs(prev_loss - loss) < self.epsilon:
                break
            prev_loss = loss
        return self


    # Предсказание для новых объектов
    def predict(self, X):
        X = self._to_2d(X)
        return X @ self.weights + self.bias

    def mse(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        pred = self.predict(X)
        return float(np.mean((pred - y)**2))

    def r2_score(self, X, y):
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        pred = self.predict(X)
        ss_res = np.sum((y - pred)**2)
        ss_tot = np.sum((y - y.mean())**2)
        return float(1 - ss_res / (ss_tot + 1e-12))


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
# импортируем os: настраиваем окружение/потоки
import os
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "4")
os.environ.setdefault("MKL_NUM_THREADS", "4")
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "4")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "4")

# импортируем pandas: работа с таблицами (DataFrame)
import pandas as pd
# импортируем numpy: работа с массивами и матрицами
import numpy as np

if 'df' in globals():
    # Удаляем скейленные дубликаты — в дальнейшем мы и так будем масштабировать
    scaled_cols = [c for c in df.columns if c.endswith('_scaled')]
    if scaled_cols:
        df = df.drop(columns=scaled_cols) # намеренно перезаписываем df, чтобы очистить память
    
    # Целочисленные признаки (этажа, код типа здания и т.п.) уменьшаем до более узких типов
    for c in ['rooms','level','levels','building_type']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='integer') # меньше памяти, значения не теряем

    # Вещественные признаки переводим в float32 — точности достаточно, памяти меньше
    for c in ['area','kitchen_area','dist_to_moscow_km','dist_to_spb_km']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='float')

    # Булевы флаги храним как uint8
    for c in ['is_Moscow','is_Saint_Peterburg','in_20km_Moscow','in_20km_SPB']:
        if c in df.columns:
            df[c] = df[c].astype('uint8')

# Приводим признаки к одному формату (float32) и кодируем категориальные (building_type) через One-Hot
# Делим на train/test (80/20) один раз
# Для KNN берём небольшую подвыборку (например, 60к/15к), чтобы не грузить устройство при работе
# Для sklearn-KNN используем тот же сабсэмпл и kd_tree
# Линейные модели считаем на полном train/test — они легкие
# Благодарая таким облегчениям работа будет проходить быстро
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor as SKKNN
from sklearn.linear_model import LinearRegression as SKLR

def to_f32(a):
    # Если матрица разреженная, то превращаем в плотную и в float32, иначе просто меняем тип
    return a.toarray().astype(np.float32) if hasattr(a, 'toarray') else a.astype(np.float32)

target = 'price'
# В некоторых ячейках же делали One-Hot и в df появились столбцы building_type_*
# Если исходного 'building_type' больше нет, то используем готовые OHE-колонки.
cat_cols = ['building_type'] if 'building_type' in df.columns else []
ohe_ready = [c for c in df.columns if c.startswith('building_type_')]
# Числовые признаки
num_candidates = ['rooms','level','levels','area','kitchen_area', 'days_since_start',
                  'days_to_end','dist_to_moscow_km','dist_to_spb_km']
num_cols = [c for c in num_candidates if c in df.columns]

# Для поддержания скорости делаем fit_transform один раз на всём df
if not cat_cols and ohe_ready:
    # Если OHE уже сделан, то просто скейлим числа и склеиваем с готовыми дамми
    scaler = StandardScaler()
    X_num = scaler.fit_transform(df[num_cols]) if num_cols else np.empty((len(df),0))
    X = np.hstack([to_f32(X_num), df[ohe_ready].to_numpy(np.float32)])
else:
    # Кодируем building_type и масштабируем числовые в одном ColumnTransformer
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float32)
    ct = ColumnTransformer([('cat', ohe, cat_cols), ('num', StandardScaler(), num_cols)])
    X = to_f32(ct.fit_transform(df[cat_cols + num_cols]))

y = df[target].to_numpy(np.float32)

# Делим на обучение и тест, перемешиваем, фиксируем random_state для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = to_f32(X_train); X_test = to_f32(X_test)

# Подвыборка для KNN
rng = np.random.default_rng(42)
max_train = min(60000, len(X_train))
max_test  = min(15000,  len(X_test))
tr_idx = rng.choice(len(X_train), size=max_train, replace=False)
te_idx = rng.choice(len(X_test),  size=max_test,  replace=False)
Xtr_s, ytr_s = X_train[tr_idx], y_train[tr_idx]
Xte_s, yte_s = X_test[te_idx],  y_test[te_idx]

# Обучаем наш KNN
knn_my = KNNRegressor(n_neighbors=5, metric='euclidean', weights='distance')
knn_my.fit(Xtr_s, ytr_s)
y_pred_knn_my = knn_my.predict(Xte_s, block_size=512)

# Линейная регрессия, считается на полном train/test
lin_my = LinearRegression(learning_rate=0.1, optimization='AdaGrad', max_iter=2000, epsilon=1e-8)
lin_my.fit(X_train, y_train)
y_pred_lin_my = lin_my.predict(X_test)

# KNN из sklearn на той же подвыборке
# Параметр algorithm='kd_tree' ускоряет поиск соседей на числовых признаках
knn_sk = SKKNN(n_neighbors=5, weights='distance', algorithm='kd_tree', leaf_size=30, n_jobs=-1)
knn_sk.fit(Xtr_s, ytr_s)
y_pred_knn_sk = knn_sk.predict(Xte_s)

# Линейная регрессия из sklearn
lr_sk = SKLR().fit(X_train, y_train)
y_pred_lr_sk = lr_sk.predict(X_test)

# MSE, MAE, RMSE и R^2
def metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = float(np.sqrt(mse))
    r2 = r2_score(y_true, y_pred)
    return mse, mae, rmse, r2

rows = [
    ('KNN',) + metrics(yte_s, y_pred_knn_my),
    ('KNN (sklearn)',) + metrics(yte_s, y_pred_knn_sk),
    ('Linear',) + metrics(y_test, y_pred_lin_my),
    ('Linear (sklearn)',) + metrics(y_test, y_pred_lr_sk),
]
res = pd.DataFrame(rows, columns=['Модель','MSE','MAE','RMSE','R2']).sort_values('RMSE')
display(res)


,Модель,MSE,MAE,RMSE,R2
1,KNN (sklearn),1.050840e+15,3.093999e+06,3.241666e+07,-2.422727
0,KNN,1.050952e+15,3.094137e+06,3.241839e+07,-2.423091
3,Linear (sklearn),1.864597e+17,4.998222e+06,4.318098e+08,0.000368
2,Linear,1.865777e+17,7.017058e+06,4.319464e+08,-0.000264
